In [50]:
import json
import csv
import datetime as dt
import time
import heapq


### Classes

class Edge:

    def __init__(self, start_node_id: int = None, end_node_id: int = None, day_type: str = None, hour: int = None, length: int = None, max_speed: float = None, time: float = None) -> None:
        self.start_node_id = start_node_id
        self.end_node_id = end_node_id
        self.day_type = day_type
        self.hour = hour
        self.length = length
        self.max_speed = max_speed
        self.time = time

class Node:

    def __init__(self, id: int = None, lat: str = None, lon: str = None) -> None:
        self.id = id
        self.lat = float(lat)
        self.lon = float(lon)
        self.neighbors = {} # {end_node_id: Edge}
        self.drivers = []

    def shortest_path(self, end_node):
        distances = {}
        distances[self.id] = 0
        pq = [(0, self)]
        while pq:
            (dist, current_node) = heapq.heappop(pq)
            
            if current_node.id == end_node.id:
                return dist
            
            if current_node in distances and dist > distances[current_node.id]:
                continue
            
            for edge_id, edge in current_node.neighbors.items():
                neighbor = nodes_id[edge.end_node_id]
                distance = dist + edge.time
                if neighbor.id not in distances or distance < distances[neighbor.id]:
                    distances[neighbor.id] = distance
                    heapq.heappush(pq, (distance, neighbor))
        return distances

class Driver:

    def __init__(self, timestamp: str = None, lat: str = None, lon: str = None) -> None:
        self.time = time.mktime(dt.datetime.strptime(timestamp, date_format).timetuple())
        self.lat = float(lat)
        self.lon = float(lon)
        self.node = nodes_gps[(self.lon, self.lat)]
        

class Passenger:

    def __init__(self, timestamp: str = None, s_lat: str = None, s_lon: str = None, e_lat: str = None, e_lon: str = None) -> None:
        self.time = time.mktime(dt.datetime.strptime(timestamp, date_format).timetuple())
        self.start_lat = float(s_lat)
        self.start_lon = float(s_lon)
        self.end_lat = float(e_lat)
        self.end_lon = float(e_lon)
        self.start_node = nodes_gps[(s_lon, s_lat)]
        self.end_node = nodes_gps[(e_lon, e_lat)]

class Ride:

    def __init__(self, start_time: str = None, end_time: str = None, driver: int = None, passenger: int = None, start_lat: float = None, start_lon: float = None, end_lat: float = None, end_lon: float = None) -> None:
        self.start_time = start_time # convert to dt.datetime obj
        self.end_time = end_time # convert to dt.datetime obj
        self.driver = driver
        self.passenger = passenger
        self.start_lat = start_lat
        self.start_lon = start_lon
        self.end_lat = end_lat
        self.end_lon = end_lon

### Graph Construction

with open('adjacency.json', 'r') as g:
    adjList = json.load(g)

with open('node_data.json', 'r') as n:
    nodeList = json.load(n)

nodes_id = {}
nodes_gps = {} # (lon, lat) -> Node()
for node_id in nodeList:
    nodes_id[int(node_id)] = Node(id = node_id, lat = nodeList[node_id]['lat'], lon = nodeList[node_id]['lon'])
    nodes_gps[(nodes_id[int(node_id)].lon, nodes_id[int(node_id)].lat)] = nodes_id[int(node_id)]

for start_node in adjList:
    node = nodes_id[int(start_node)]
    for end_node in adjList[start_node]:
        for route in adjList[start_node][end_node]:
            if int(end_node) not in node.neighbors:
                node.neighbors[int(end_node)] = [Edge(start_node_id = start_node, end_node_id = end_node, **adjList[start_node][end_node][route])]
            else:
                node.neighbors[int(end_node)].append(Edge(start_node_id = start_node, end_node_id = end_node, **adjList[start_node][end_node][route]))


### Reading Drivers and Passengers

date_format = "%m/%d/%Y %H:%M:%S"

drivers = []
with open('drivers.csv', 'r') as d:
    _ = d.readline()
    d_reader = csv.reader(d)
    for d in d_reader:
        drivers.append(Driver(*d))

passengers = []
with open('passengers.csv', 'r') as p:
    _ = p.readline()
    p_reader = csv.reader(p)
    for p in p_reader:
        passengers.append(Passenger(*p))


TypeError: list indices must be integers or slices, not dict

In [ ]:
from collections import deque

In [ ]:
i, j = 0, 0 # i is index for passengers, j is index for drivers
time = 0
q = deque() # queue of drivers
ongoing_rides = [] # heap for ongoing rides
passenger_q = deque() # queue of passengers

time_before_assign, time_to_passenger, driver_idle_time = 0, 0, 0

while i < len(passengers):
    passenger_q.append((passengers[i], passengers[i].time))
    time = passenger_q[-1].time # update time

    # adds new drivers and completed rides back into queue in order by time
    while (j < len(drivers) and drivers[j].time < time) or (ongoing_rides and ongoing_rides[0][1] < time):
        if drivers and (not ongoing_rides or drivers[j] < ongoing_rides[0][1]):
            q.append((drivers[j], drivers[j].time))
            j += 1
        else:
            t, driver = heapq.heappop(ongoing_rides)
            q.append((driver, t))

    if not q:
        i += 1
        continue

    while passenger_q and q:
        new_passenger, passenger_time = passenger_q.popleft()
        paired_driver, driver_time = q.popleft()

        time_before_assign += max(driver_time, passenger_time) - passenger_time
        driver_idle_time += max(driver_time, passenger_time) - driver_time
    
        # how long ride takes
        dt = paired_driver.node.shortest_path(new_passenger.start_node) * 3600

        time_to_passenger = dt

        dt += new_passenger.start_node.shortest_path(new_passenger.end_node) * 3600

        # push ride to heap, update location of driver
        heapq.heappush(ongoing_rides, (max(passenger_time, driver_time) + dt, paired_driver))
        paired_driver.node = new_passenger.end_node

    i += 1

print(f'Average time before ride is assigned: {time_before_assign/len(passengers)}')
print(f'Average time for driver to reach passenger: {time_to_passenger/len(passengers)}')
print(f'Average time driver is idle: {time_before_assign/len(drivers)}')



AttributeError: 'tuple' object has no attribute 'time'